Relevant imports and initialization code.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import random
from numpy import savetxt

seed = 25
np.random.seed(seed)

This section reads in our data from our Spotify track dataset. This dataset has a large amount of data that's captured per track.

Let's read in our other dataset that contains user created playlist with songs with them. This will be crucial to generating a desired y label that we will want our neural network to train on.

You'll need to download the spotify song dataset and the spotify recommendator challenge dataset from https://www.kaggle.com/datasets/zaheenhamidani/ultimate-spotify-tracks-db/ and https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge respectively and put them into the dataset folder in order to run this code.

It's not in the git repo since it's too large.

In [2]:
sample_size = 4000
original_df = pd.read_csv('./datasets/spotify_songs.csv')
original_df.drop_duplicates('track_id', inplace=True)
original_df = original_df[['genre','artist_name','track_name','track_id','popularity','acousticness','danceability','duration_ms','energy','instrumentalness','key','liveness','loudness','mode','speechiness','tempo','time_signature','valence']]
# we only want a certain number of columns. Reducing unneeded features will improve performance.


playlist_df = pd.read_json('./datasets/challenge_set.json')
playlist_series = playlist_df['playlists']

Note that the code before does not output any information about the genre or track id. This is because in the dataset they are string values.

This code section rearranges the dataset to be more compatible with machine learning. One aspect of this is converting unique string values into a numeric equivalent. The Tensorflow normalization layer will handle proper normalization after that.

In [4]:
def encode_values(col):
    unique_items = col.unique().tolist()
    items_to_encoded = {x: i for i, x in enumerate(unique_items)}
    encoded_to_items = {i: x for i, x in enumerate(unique_items)}
    return (items_to_encoded, encoded_to_items)

genre_items_to_encoded, genre_encoded_to_items = encode_values(original_df['genre'])
original_df['genre'] = original_df['genre'].map(genre_items_to_encoded)

artist_name_items_to_encoded, artist_name_encoded_to_items = encode_values(original_df['artist_name'])
original_df['artist_name'] = original_df['artist_name'].map(artist_name_items_to_encoded)

track_name_items_to_encoded, track_name_encoded_to_items = encode_values(original_df['track_name'])
original_df['track_name'] = original_df['track_name'].map(track_name_items_to_encoded)

track_id_items_to_encoded, track_id_encoded_to_items = encode_values(original_df['track_id'])
original_df['track_id'] = original_df['track_id'].map(track_id_items_to_encoded)

key_items_to_encoded, key_encoded_to_items = encode_values(original_df['key'])
original_df['key'] = original_df['key'].map(key_items_to_encoded)

mode_items_to_encoded, mode_encoded_to_items = encode_values(original_df['mode'])
original_df['mode'] = original_df['mode'].map(mode_items_to_encoded)

time_signature_items_to_encoded, time_signature_encoded_to_items = encode_values(original_df['time_signature'])
original_df['time_signature'] = original_df['time_signature'].map(time_signature_items_to_encoded)

df = original_df.copy()
df = df[sample_size:sample_size*2]
df.describe()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4.000000e+03,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,3.449750,683.860750,5574.526250,5999.500000,46.171250,0.221302,0.558485,2.257594e+05,0.671749,0.033769,5.421250,0.197724,-6.972861,0.249000,0.072259,123.099086,0.146500,0.487848
std,0.497531,441.645122,1342.866369,1154.844867,7.734816,0.264964,0.134234,6.062448e+04,0.211033,0.131651,3.509113,0.161859,3.070843,0.432488,0.079433,30.358727,0.524029,0.225759
min,3.000000,40.000000,7.000000,4000.000000,0.000000,0.000003,0.127000,2.671700e+04,0.018800,0.000000,0.000000,0.021400,-25.669000,0.000000,0.022400,47.811000,0.000000,0.034200
25%,3.000000,323.750000,4680.750000,4999.750000,39.000000,0.010675,0.472000,1.914582e+05,0.515000,0.000000,2.000000,0.099300,-8.506000,0.000000,0.031800,97.990500,0.000000,0.315000
50%,3.000000,534.000000,5653.500000,5999.500000,48.000000,0.093400,0.558500,2.170065e+05,0.702500,0.000010,5.000000,0.132000,-6.347000,0.000000,0.041300,120.130500,0.000000,0.476500
75%,4.000000,1018.000000,6639.250000,6999.250000,52.000000,0.376000,0.649000,2.499168e+05,0.848000,0.000937,8.000000,0.254000,-4.744250,0.000000,0.072400,144.051000,0.000000,0.660250
max,4.000000,1696.000000,7634.000000,7999.000000,72.000000,0.974000,0.936000,1.355938e+06,0.998000,0.949000,11.000000,0.994000,-0.259000,1.000000,0.918000,213.788000,3.000000,0.975000


Here we build a pseudo histogram where we capture the frequency in which a particular song is in a playlist with another song. In training, this will generate the y label based on how frequent a song is found with another one.

In [5]:
train_indices = int(0.9 * df.shape[0])
train_df = df.iloc[:train_indices]
count = 0

# encoded_track_histogram = {int(row['track_id']): set() for index, row in df.iterrows()}
# for playlist in playlist_series:
#     playlist_tracks = [track['track_uri'].split(':')[2] for track in playlist['tracks']]
#     included_tracks = [track for track in playlist_tracks if track in track_id_items_to_encoded]
#     encoded_tracks = [track_id_items_to_encoded[track] for track in included_tracks]
    
#     if (len(encoded_tracks) > 1):
#         for x in encoded_tracks:
#             for y in encoded_tracks:
#                 encoded_track_histogram[x].add(y)
#                 encoded_track_histogram[y].add(x)
#                 count = count + 2

x_data = df.values.astype(np.float32)
y_data = []

normalized_df = (df - df.mean()) / df.std()
normalized_train_df = normalized_df.iloc[:train_indices]

print(len(list(df.iterrows())))
print(df.to_numpy().size)
print(normalized_df.to_numpy().size)
#print(len(encoded_track_histogram))

for index, row in normalized_df.iterrows():
    probability = np.zeros(train_indices)
    distances = {id: np.sqrt(np.sum((np.array(row) - np.array(other_row)) ** 2)) for id, other_row in enumerate(normalized_train_df.to_numpy())}
    sorted_distances = sorted(distances.items(), key=lambda x: x[1])
    closest = sorted_distances[1:6]

    histogram_data = [datapoint[0] for datapoint in closest] #+ [datapoint for datapoint in encoded_track_histogram[index] if datapoint < len(probability)]

    probability[histogram_data[0]] = 0.5
    probability[histogram_data[1]] = 0.3
    probability[histogram_data[2]] = 0.1
    probability[histogram_data[3]] = 0.08
    probability[histogram_data[4]] = 0.02
    # for histogram_datapoint in histogram_data:
    #     probability[histogram_datapoint] = probability[histogram_datapoint] + (1 / len(histogram_data))

    y_data.append(probability)



y_data = np.array(y_data)
test = y_data.sum(axis=1)
savetxt('test.txt', test)
print(count)

x_train = x_data[:train_indices]
x_test = x_data[train_indices:]
y_train = np.array(y_data)[:train_indices]
y_test = np.array(y_data)[train_indices:]

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


4000
72000
72000
0
(3600, 18)
(400, 18)
(3600, 3600)
(400, 3600)


This section builds the model.

In [9]:
def build_and_compile_model(features, num_classes):
    normal_layer = tf.keras.layers.Normalization(axis=-1)
    normal_layer.adapt(features)

    model = keras.Sequential([
      normal_layer,
      keras.layers.Dense(2000, activation='relu'),
      keras.layers.Dense(2000, activation='relu'),
      keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [10]:

model = build_and_compile_model(x_train, len(x_train))
model.summary()

model.fit(x_train, y_train, epochs=50, batch_size=500, verbose=2) #  validation_data=(x_test, y_test)
scores = model.evaluate(x_test, y_test)
print('Baseline error: %2.f%%' % (100 - scores[1] * 100))



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 18)                37        
 tion)                                                           
                                                                 
 dense_3 (Dense)             (None, 2000)              38000     
                                                                 
 dense_4 (Dense)             (None, 2000)              4002000   
                                                                 
 dense_5 (Dense)             (None, 3600)              7203600   
                                                                 
Total params: 11243637 (42.89 MB)
Trainable params: 11243600 (42.89 MB)
Non-trainable params: 37 (152.00 Byte)
_________________________________________________________________
Epoch 1/50
8/8 - 1s - loss: 8.0366 - accuracy: 0.0119 - 1s/epoch - 144ms/st

In [120]:
def print_row_details(data):
    print(genre_encoded_to_items[data['genre']])
    print(artist_name_encoded_to_items[data['artist_name']])
    print(track_name_encoded_to_items[data['track_name']])
    print(track_id_encoded_to_items[data['track_id']])
    print(key_encoded_to_items[data['key']])
    print(mode_encoded_to_items[data['mode']])

index = random.randrange(len(df))
track_id = '1EYl3ASOxlK4Fk4Q1bhDh4'
#data_to_predict = original_df.iloc[index]
data_to_predict = original_df.iloc[track_id_items_to_encoded[track_id]]

print('Selected data point')
print_row_details(data_to_predict)

prediction = model.predict(data_to_predict.to_numpy().astype(np.float64))

# print(prediction.shape)
# with np.printoptions(threshold=np.inf):
#     print(prediction)
# savetxt('data.npy', prediction[0])

for row in prediction:
    prediction_index = row.argmax()
    print(row[prediction_index])
    print('Prediction:')
    print_row_details(df.iloc[prediction_index])
    print('')


Selected data point
Country
Johnny Cash
The One on the Right Is on the Left
1EYl3ASOxlK4Fk4Q1bhDh4
A
Major
1/1 [==============================] - 0s 17ms/step
0.18193291
Prediction:
Country
The Doobie Brothers
Another Park, Another Sunday - 2006 Remaster
5dEOntLHunr3jYzS1XBNmk
A
Major

